In [1]:
from tardis.atomic import AtomData
import numpy as np
import pandas as pd

atomic_dataset = AtomData.from_hdf5('kurucz_cd23_chianti_H_He.h5')

tardis.atomic - INFO - Read Atom Data with UUID=5ca3035ca8b311e3bb684437e69d75d7 and MD5=21095dd25faa1683f4c90c911a00c3f8


In [2]:
def get_index(ele):
    index=-1
    for atomic_no,row in atomic_dataset.atom_data.iterrows():
        if ele in row['name']:
            index = atomic_no
            break
    return index

In [45]:
def extract_block(f):
    qty = []
    for line in f:
        items = line.split()
        if items:
            qty.extend(np.array(items).astype(np.float64))
        else:
            break
    qty = np.array(qty)
    return qty

In [50]:
def parse_file(fname):
    with open(fname,'r') as f:
        for line in f:
            items = line.split()
            n = len(items)
            
            if 'data points' in line:
                no_of_shells = int(items[n-1])
                df = pd.DataFrame(columns=np.arange(int(items[n-1])),
                         index=pd.Index([],
                                        name='element'),
                         dtype=np.float64)            
            if 'Velocity' in line:
                velocity = extract_block(f)
            if 'Density' in line:
                density = extract_block(f)
            if 'Electron density' in line:
                elec_density = extract_block(f)
            if 'mass fraction\n' in line:
                    abundances = []
                    element_string = items[0]
                    atomic_no = get_index(element_string.capitalize())
                    element_symbol = atomic_dataset.atom_data.loc[atomic_no]['symbol']
                    
                    #Its a Isotope
                    if n==4:
                        element_symbol+=items[1]
                    
                    for line in f:
                        items =line.split()
                        if items:
                            abundances.extend( np.array(items).astype(np.float64))
                        else:
                            break
                    
                    df.loc[element_symbol] = abundances
    print elec_density
    return df.transpose()
                

In [51]:
df = parse_file('DDC15_SN_HYDRO_DATA_0.976d')
#df.to_csv('test.csv',index=False,sep=' ')

[  4.70364060e+04   4.71124880e+04   4.76518420e+04   4.83089110e+04
   4.93082900e+04   5.08966210e+04   5.37999370e+04   6.13401300e+04
   7.20278070e+04   8.71029750e+04   1.08745060e+05   1.39554170e+05
   1.83737430e+05   2.47223360e+05   3.38391730e+05   4.69982550e+05
   6.59094780e+05   9.32716610e+05   1.32840960e+06   1.90293130e+06
   2.73615280e+06   3.94278520e+06   5.68465750e+06   8.18390870e+06
   1.17596450e+07   1.68244670e+07   2.39401650e+07   3.38978510e+07
   4.82262960e+07   6.76849540e+07   9.11996970e+07   1.15036740e+08
   1.38209070e+08   1.71347700e+08   2.22237930e+08   2.79449510e+08
   3.37667470e+08   5.13524160e+08   1.01221150e+09   2.11526190e+09
   3.65566270e+09   5.89768600e+09   8.96286570e+09   1.26015390e+10
   1.68887030e+10   2.23625640e+10   3.06896830e+10   4.34738250e+10
   6.03532270e+10   8.28886440e+10   1.19219490e+11   1.84117150e+11
   2.80771530e+11   3.61764010e+11   4.75848880e+11   6.84759360e+11
   1.06128230e+12   1.86431720e+12

TypeError: Required argument 'object' (pos 1) not found